In [1]:
try:
    import customerLibrary
except ImportError:
    customerLibrary = None


In [2]:
class Custom:
    if customerLibrary is not None:
        hasLibrary = True
    else:
        hasLibrary = False


In [4]:
Custom.hasLibrary 

False

In [5]:
class Egg(int):
    spam = 'egg'

In [6]:
Egg

__main__.Egg

## 元类

In [12]:
class PluginMount(type):
    """
    Place the metaclass (元類) on any standard python to turn it into a plugin mount point.
    All subclass will be automatically registered as plugins.
    在python中， 元类是创建类的类。当你定义一个元类的时，__init__(cls) 方法的第一个参数是cls，而不是self, 
    这是因为__init__方法实在创建类的过程中被调用而不是在类的实例化过程中。 
    
    """
    def __init__(cls, name, bases, attrs):
        if not hasattr(cls, 'plugins'):
            cls.plugins = []
        else:
            cls.plugins.append(cls)

In [13]:
class InputValidator(metaclass=PluginMount):
    """
    A plugin mount for input validation.
    Supported plugins must provide a validate(self, input) method, 
    which receives input as string and raises a ValueError if the 
    input was invalid. if the input was properly valid, it should just
    return without error. any return value will be ignored.

    """
    def validator(self, input):
        """
        the default implementation raise a NotImpementationError
        to ensure that any subclasses must override this method.
        :param input: 
        :return: 
        """
        raise NotImplementedError
    

In [14]:
class ASCIIValidator(InputValidator):
    """
    validate the input only consists of valid ASCII characters.
    """
    def validator(self, input):
        """
        if the encoding operation fails, str.encode() will raise a UncodeDecoeError
        :param input: 
        :return: 
        """
        input.encode('ascii')
        

In [15]:
InputValidator.plugins

[__main__.ASCIIValidator]

In [19]:
class MyMeta(type):
    def __init__(cls, name, bases, attrs):
        print(f'Creating class: {name}')
        """
        super().__init__(name, bases, attrs)的作用是调用父类的__init__方法，
        确保父类（在这里是type类）的初始化逻辑被执行。这是一个重要的步骤，通常在自定义类时需要确保基类的初始化代码能够正确执行，以便为新创建的类提供必要的属性和行为。
        
        1， 确保基类初始化：当你定义一个元类时，它是继承自type类的。调用super().__init__(name, bases, attrs)可以确保type类的初始化逻辑被调用，这样Python的类系统能够为新类设置好必要的基础设施。

        2， 维护多重继承：在Python中，super()可以帮助你处理多重继承的情况。它会按照方法解析顺序（MRO）来查找下一个合适的类，从而确保正确调用各个基类的方法。这一特性在复杂的继承结构中尤其重要。
        
        3， 扩展功能：虽然在这个简单的元类示例中，super().__init__只是调用了基础类的初始化方法，但在更复杂的元类中，你可能需要在重写的__init__方法中添加额外的逻辑，同时也希望保留基类的初始化行为。通过使用super()可以很方便地做到这一点。
        
        总之，super().__init__(name, bases, attrs)的调用是为了确保元类的父类type的初始化过程能够被正确执行，从而使得创建的类能够正常工作，并具备Python类的基本特性。

        """
        super().__init__(name, bases, attrs)
        

In [20]:
class MyClass(metaclass=MyMeta):
    pass


Creating class: MyClass


-------------

###   属性
    1， 类属性
    2， 对象属性
    3,  描述符
    

In [51]:
class Person:
    species = 'Hello Cat!!'  ## 类属性
    
    def __init__(self, name):
        self.name = name
print(Person.species)

Hello Cat!!


In [52]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              'species': 'Hello Cat!!',
              '__init__': <function __main__.Person.__init__(self, name)>,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

In [54]:
person = Person('dong')

In [55]:
person.__dict__

{'name': 'dong'}

            ---- 描述符

In [57]:
class NameDescriptor:
    def __init__(self, name):
        self.name = name
    
    def __get__(self, instance, owner):
        """
        :param instance:  拥有描述符的实例（在这个例子中是 obj）
        :param owner:  拥有描述符的类（在这个例子中是 MyClass）
        :return: 
        """    
        print(f'Getting: {self.name}')
        return instance.__dict__[self.name]
    
    def __set__(self, instance, value): 
        print(f'Setting: {self.name} to {value}') 
        if not isinstance(value, str):
            raise ValueError('Name must be a string')
        instance.__dict__[self.name] = value
    
    def __delete__(self, instance):
        del instance.__dict__[self.name]
    

In [58]:
class MyClass:
    name = NameDescriptor('name')
    
    def __init__(self, name):
        self.name = name
        
            

In [60]:
dong = MyClass('dong')

Setting: name to dong


In [61]:
dong.name

Getting: name


'dong'

- --------------------

### 如果在编写程序的时候， 不知道成员变量的名称，可以使用内置的getAttr()函数为名称提供变量。  

In [6]:
class MyClass:
    def __init__(self):
        self.value = 43
        
    def __getattr__(self, item):
        return f'{item} attibute is not present.'


In [7]:
obj = MyClass()

In [23]:
str(obj)

'<__main__.MyClass object at 0x0000020136ADEE10>'

In [8]:
getattr(obj, 'value')

43

In [9]:
getattr(obj, 'non_existed_value')


'non_existed_value attibute is not present.'

In [10]:
getattr(obj, 'non_existed_value', 34)

'non_existed_value attibute is not present.'

In [11]:
obj.non_existed_attr

'non_existed_attr attibute is not present.'

In [12]:
obj.value

43

In [13]:
class AttibuteDict(dict):
    def __getattr__(self, item):
        return self[item]

In [15]:
d = AttibuteDict(spam='eggs')

In [16]:
d['spam']

'eggs'

In [21]:
d

{'spam': 'eggs'}

In [25]:
class MyClass:
    def __init__(self, value):
        self.value = value

    def __repr__(self):
        return f"MyClass(value={self.value!r})"

obj = MyClass(42)


In [26]:
obj

MyClass(value=42)